In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# sk learn imports for the ML model
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR


from sklearn.metrics import mean_squared_error

## MLFLow Import

In [2]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location=('/workspaces/MLOps-Model-Development-to-Production-deployment/Experiment '
 'Tracking using MLFlow/mlruns/1'), creation_time=1749424860439, experiment_id='1', last_update_time=1749424860439, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
df = pd.read_parquet('/workspaces/MLOps-Model-Development-to-Production-deployment/Data/green_tripdata_2025-01.parquet')
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-01-01 00:03:01,2025-01-01 00:17:12,N,1.0,75,235,1.0,5.93,24.70,...,0.5,6.80,0.00,NaN,1.0,34.00,1.0,1.0,0.00,0.0
1,2,2025-01-01 00:19:59,2025-01-01 00:25:52,N,1.0,166,75,1.0,1.32,8.60,...,0.5,0.00,0.00,NaN,1.0,11.10,2.0,1.0,0.00,0.0
2,2,2025-01-01 00:05:29,2025-01-01 00:07:21,N,5.0,171,73,1.0,0.41,25.55,...,0.0,0.00,0.00,NaN,1.0,26.55,2.0,2.0,0.00,0.0
3,2,2025-01-01 00:52:24,2025-01-01 01:07:52,N,1.0,74,223,1.0,4.12,21.20,...,0.5,6.13,6.94,NaN,1.0,36.77,1.0,1.0,0.00,0.0
4,2,2025-01-01 00:25:05,2025-01-01 01:01:10,N,1.0,66,158,1.0,4.71,33.80,...,0.5,7.81,0.00,NaN,1.0,46.86,1.0,1.0,2.75,0.0


## Preprocessing of data

In [4]:
df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [5]:
train_dicts = df[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred)

44.94737843983856

In [6]:
def preprocess_df(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [7]:
df_train = preprocess_df('/workspaces/MLOps-Model-Development-to-Production-deployment/Data/green_tripdata_2025-01.parquet')
df_val = preprocess_df('/workspaces/MLOps-Model-Development-to-Production-deployment/Data/green_tripdata_2025-02.parquet')

print(df_train.shape, df_val.shape)

(46307, 22) (44218, 22)


In [8]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [10]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred)

37.19536000471952

In [11]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [12]:
## define a new run and put in the ml flow run

with mlflow.start_run():

    mlflow.set_tag("developer", "keyush")

    ## we can also log a different params
    mlflow.log_param("train-dataPath", "/workspaces/MLOps-Model-Development-to-Production-deployment/Data/green_tripdata_2025-01.parquet")
    mlflow.log_param("val-dataPath", "/workspaces/MLOps-Model-Development-to-Production-deployment/Data/green_tripdata_2025-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    mse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("mse", mse)

In [13]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [14]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

### fmin()
This is the main minimization function.
You pass a loss function (e.g., RMSE), a search space, and an algorithm.
It returns the best set of hyperparameters.

#### tpe (Tree-structured Parzen Estimator)
A Bayesian optimization algorithm.
Learns from previous trials to decide what parameters to try next (smarter than random search).

#### hp

Used to define the search space of hyperparameters.
Examples:

hp.uniform('learning_rate', 0.01, 0.3)
hp.choice('max_depth', [3, 5, 7, 10])

#### scope (from hyperopt.pyll)
Used to apply transformations to sampled hyperparameters.

Example:

scope.int(hp.quniform('max_depth', 3, 10, 1))
This means: sample integers from 3 to 10 (step 1).


In [21]:
import mlflow.xgboost


def objective_eval(params):
    with mlflow.start_run():
        # mlflow.set_tag("model", "xgboost")
        mlflow.set_tag("model", "xgboost_2")
        mlflow.log_params(params)
        xg = xgb.train(
            params,
            dtrain = train,
            num_boost_round = 1000, ## there will be 1000 iterations
            evals=[(valid, 'validation')],
            early_stopping_rounds=50 ## if there is no improvement then the model will stop. That is why you will less than 1000 rounds below in 
            ## each of the runs which is like an epoch where a new model is instantiated and run!
        ) 
        y_pred = xg.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        mlflow.xgboost.log_model(xg, artifact_path="models_mlflow")

    return {"loss": rmse, "status": STATUS_OK}

In [22]:
## hp helps us to modify the parameters, 

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), ## here we find a value between e^(-3) whch is 0.05 and exp(0) which is 1
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear', ## this is a reg problem and the scope of the target is continuous
    'seed': 42
}

best_result = fmin(
    fn=objective_eval,
    space=search_space,
    algo=tpe.suggest, ## tpe is the algorithm that will be used to search the best set of hyper params!
    max_evals=50,
    trials=Trials() ## this will help in logging the metrics
)

# from math import exp
# exp(-3)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/conda/envs/mlflow-exp/lib/python3.10/site-packages/xgboost/callback.py:386: UserWarning: [19:01:31] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.62174                           
[1]	validation-rmse:8.19759                           
[2]	validation-rmse:7.82780                           
[3]	validation-rmse:7.50611                           
[4]	validation-rmse:7.22790                           
[5]	validation-rmse:6.98744                           
[6]	validation-rmse:6.78090                           
[7]	validation-rmse:6.60383                           
[8]	validation-rmse:6.45268                           
[9]	validation-rmse:6.32340                           
[10]	validation-rmse:6.21304                          
[11]	validation-rmse:6.11983                          
  0%|          | 0/50 [00:03<?, ?trial/s, best loss=?]

[12]	validation-rmse:6.04037                          
[13]	validation-rmse:5.97304                          
[14]	validation-rmse:5.91571                          
[15]	validation-rmse:5.86670                          
[16]	validation-rmse:5.82533                          
[17]	validation-rmse:5.79001                          
[18]	validation-rmse:5.75916                          
[19]	validation-rmse:5.73213                          
[20]	validation-rmse:5.70922                          
[21]	validation-rmse:5.68995                          
[22]	validation-rmse:5.67312                          
[23]	validation-rmse:5.65843                          
[24]	validation-rmse:5.64453                          
[25]	validation-rmse:5.63171                          
[26]	validation-rmse:5.62186                          
[27]	validation-rmse:5.61236                          
[28]	validation-rmse:5.60423                          
[29]	validation-rmse:5.59653                          
[30]	valid

/opt/conda/envs/mlflow-exp/lib/python3.10/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [19:03:17] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)

2025/06/09 19:03:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



...

 98%|█████████▊| 49/50 [34:27<00:33, 33.65s/trial, best loss: 5.30110649565218]

/opt/conda/envs/mlflow-exp/lib/python3.10/site-packages/xgboost/callback.py:386: UserWarning: [19:35:58] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:7.36505                                                    
[1]	validation-rmse:6.45505                                                    
[2]	validation-rmse:6.00475                                                    
[3]	validation-rmse:5.78528                                                    
[4]	validation-rmse:5.67402                                                    
[5]	validation-rmse:5.60719                                                    
[6]	validation-rmse:5.56865                                                    
[7]	validation-rmse:5.54574                                                    
[8]	validation-rmse:5.53078                                                    
[9]	validation-rmse:5.51774                                                    
[10]	validation-rmse:5.50732                                                   
[11]	validation-rmse:5.50074                                                   
[12]	validation-rmse:5.49425            

/opt/conda/envs/mlflow-exp/lib/python3.10/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [19:36:19] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)

2025/06/09 19:36:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



100%|██████████| 50/50 [34:51<00:00, 41.83s/trial, best loss: 5.30110649565218]


### Using the best model
#### 1. Load model as a python function or PyFuncmodel
#### 2. Load model as a Xgboost model

##### We can take any of the runs or any of the variations to use it for predctions either as a Python function or as an object of XGboost or whichever model we train. As shown above in the MLflow UI, PyFuncModel allows us to use Pandas DataFrame features for its prediction method, unlike XGBoost models which requires us to convert features into DMatrix type.

In [15]:
logged_model = 'runs:/5792742ef5324a609c91590ed7b96c7f/models_mlflow' ## this is the saved run that we have chosen
loaded_model = mlflow.pyfunc.load_model(logged_model)
loaded_model ## this will give us the path of the run

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 5792742ef5324a609c91590ed7b96c7f

In [16]:
# preprocessed DataFrame with DictionaryVectorizer
# X_val = dv.transform(X_val_dict)

# convert X_val into DMatrix type
valid = xgb.DMatrix(X_val, label=y_val)
loaded_model.predict(X_val)

array([ 5.1470017, 26.56458  , 28.101507 , ..., 19.14343  , 16.275436 ,
       23.078476 ], dtype=float32)

In [17]:
# On the flip side, we can also load the model as XGBoost model:Since it is an XGBoost model, 
# we will need to use the DMatrix-typed feature inputs for our validation set:

xgb_model = mlflow.xgboost.load_model(logged_model)
xgb_model

In [ ]:
## autolog is not working as my libstdc is very old. 

# mlflow.set_tag("additional_models", "others")
# mlflow.set_tag("run_by", "keyush")

#     ## we can also log a different params

# mlflow.sklearn.autolog()

# for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

#     with mlflow.start_run():

#         mlflow.log_param("train-dataPath", "/workspaces/MLOps-Model-Development-to-Production-deployment/Data/green_tripdata_2025-01.parquet")
#         mlflow.log_param("val-dataPath", "/workspaces/MLOps-Model-Development-to-Production-deployment/Data/green_tripdata_2025-02.parquet")
#         mlflow.log_artifact("models_mlflow/preprocessor.b", artifact_path="preprocessor")

#         mlmodel = model_class()
#         mlmodel.fit(X_train, y_train)

#         y_pred = mlmodel.predict(X_val)
#         rmse = mean_squared_error(y_val, y_pred, squared=False)
#         mlflow.log_metric("rmse", rmse)


ImportError: /opt/conda/envs/mlflow-exp/bin/../lib/libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /opt/conda/envs/mlflow-exp/lib/python3.10/site-packages/scipy/fft/_pocketfft/pypocketfft.cpython-310-x86_64-linux-gnu.so)